How I produced the replays with trained baselines:
1. Clone trained baselines from https://gitlab.aicrowd.com/eric_hammy/neurips-2021-the-nethack-challenge/tree/submission-baseline-0.5B
2. Make sure the model weights are properly downloaded (they're under LFS and are ~35MB each)
3. Change line #8 in envs/wrappers.py to: `return aicrowd_gym.make("NetHackChallenge-v0", savedir='')`
4. Change NUM_EPISODES in line #38 of submission_config.py to however many you need.
5. The baselines were cloned to `/media/karolis/SSD-Ubuntu/NLE/baselines-trained`, adjust PATHs below accordingly.

In [17]:
import bz2
import os
import re

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

from tqdm.notebook import tqdm

In [18]:
PATH = '/Users/thayer/neurips-2021-nethack-starter-kit/nle_data/20210702-163442_at72fu1c'
PATH_OUT = '/Users/thayer/neurips-2021-nethack-starter-kit/parsed/parsed_large.csv'

In [19]:
# problem if the lines get cut off onto a new line
score_line_pattern = re.compile("You ((?:were |turned to )?([a-zA-Z]+)) in ([a-zA-Z ]+) on dungeon level ([0-9]+) with ([0-9]+) points")
gold_moves_line_pattern = re.compile("and ([0-9]+) pieces of gold, after ([0-9]+) moves")
exp_level_line_pattern = re.compile("You were level ([0-9]+) with a maximum of ([0-9]+) hit points when you")
vanquished_line_pattern = re.compile("([0-9]+) creatures vanquished.")
role_pattern = re.compile('role: ([a-zA-Z]+)')
race_pattern = re.compile('race: ([a-zA-Z]+)')
gender_pattern = re.compile('gender: ([a-zA-Z]+)')
alignment_pattern = re.compile('alignment: ([a-zA-Z]+)')
killer_pattern = re.compile('killed by (.+) [0-9]{4}')
headstone_pattern = re.compile('\/     PEACE([\s\S]+)\*  \*  \*')
spaces_pattern = re.compile('\s\s+')
coda_pattern1 = re.compile(b'Final Attributes:')
coda_pattern2 = re.compile(b'Final Status:')

def ttyrec_parse(f):
    d = {}
    d['path'] = path
    d['frames'] = 0
    d['kills'] = 0
    
    preamble_flag = True
    coda_flag = False
    for raw_line in f:
        d['frames'] += 1
        
        
        if preamble_flag: # our first frame contains necessary information
            line = str(raw_line)
            role_match = re.search(role_pattern, line)
            if role_match:
                role = role_match[1]
                if role == "Priest" or role == "Priestess":
                    d['role'] = "Priest/Priestess"
                elif role == "Cavewoman" or role == "Caveman":
                    d['role'] = "Caveperson"
                else:
                    d['role'] = role

            race_match = re.search(race_pattern, line)
            if race_match:
                d['race'] = race_match[1]

            gender_match = re.search(gender_pattern, line)
            if gender_match:
                d['gender'] = gender_match[1]

            alignment_match = re.search(alignment_pattern, line)
            if alignment_match:
                d['alignment'] = alignment_match[1]
            
            if role_match:
                preamble_flag = False
        
        
        coda_match1 = re.search(coda_pattern1, raw_line)
        coda_match2 = re.search(coda_pattern2, raw_line)
        if coda_match1 and coda_match2: # look for a known quantity that only appears after death to
            #(1) avoid bad matches from weird named items in game
            #(2) speed up parsing by only looking for matches in the tail of the file
            coda_flag = True
            
        if coda_flag:
            line = str(raw_line)
            score_match = re.search(score_line_pattern, line)

            if score_match:
                print(score_match)
                d['death_type'] = str(score_match[2])
                d['branch'] = str(score_match[3])
                d['depth'] = int(score_match[4])
                d['score'] = int(score_match[5])

                # this only happens in the same frame with the score line
                #there are just nasty unprintables separating them
                gold_moves_match = re.search(gold_moves_line_pattern, line)

                if gold_moves_match:
                    d['gold'] = int(gold_moves_match[1])
                    d['steps'] = int(gold_moves_match[2])
                else:
                    #print(raw_line)
                    pass
                    #assert False, "Expected gold+moves line on same frame as score line."

                exp_level_line_match = re.search(exp_level_line_pattern, line)

                if exp_level_line_match:
                    d['explevel'] = int(exp_level_line_match[1])
                    d['maxhp'] = int(exp_level_line_match[2])
                else:
                    #print(raw_line)
                    pass
                    #assert False, "Expected exp level line on same frame as score line."

            if "pieces of gold, after" in line and not score_match:
                import pdb; pdb.set_trace() # looks like we're on the score screen, but we failed to match score

            if "Vanquished creatures:" in line:
                d['kills'] = 1  # when only one line in kills, it doesn't show total separately
                # 0 kills and it doesn't show at all, hence our default of 0

            vanquished_match = re.search(vanquished_line_pattern, line) # but if many kills, we'll find a match
            if vanquished_match:
                d['kills'] = int(vanquished_match[1])

            headstone_match = re.search(headstone_pattern, line)
            if headstone_match:
                cleaner_headstone = re.sub(spaces_pattern, ' ', re.sub(r'[^\x00-\x7F]|\s|\\r|\\x1b|\[B|\[K|\|', ' ', headstone_match[1]))
                killer_match = re.search(killer_pattern, cleaner_headstone)
                if killer_match:
                    d['killer'] = killer_match[1]
                else:
                    #import pdb; pdb.set_trace()
                    pass
    return d

In [20]:
#dirs = os.listdir(PATH)

#with open(PATH_OUT, 'w') as f:
#    f.write(';'.join(FIELDS) + '\n')

#for dr in tqdm(dirs):
#    files = os.listdir(os.path.join(PATH, dr))
#    files.sort(key=lambda x: int(x.split('.')[2]))
#    for file in tqdm(files):
#        d = ttyrec_parse(os.path.join(PATH, dr, file))
#        with open(PATH_OUT, 'a') as f:
#            f.write(';'.join(str(d[f]) for f in FIELDS) + '\n')

In [21]:
dr = PATH
os.chdir(os.path.join(PATH, dr))

files = [f for f in os.listdir('.') if os.path.isfile(f)]
rows = []

for file in tqdm(files):
    path = os.path.join(PATH, dr, file)
    with bz2.BZ2File(path) as f:
        rows.append(ttyrec_parse(f))
        #if path == 'bad_path':
        #    for l in f:
        #        print(l)

  0%|          | 0/3265 [00:00<?, ?it/s]

<re.Match object; span=(3015, 3083), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(4753, 4820), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(4826, 4893), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(4528, 4595), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(4140, 4206), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(3296, 3361), match='You died in The Gnomish Mines on dungeon level 7 >
<re.Match object; span=(5616, 5682), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(4440, 4505), match='You died in The Gnomish Mines on dungeon level 5 >
<re.Match object; span=(3744, 3811), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(4178, 4246), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(4858, 4925), match='You di

<re.Match object; span=(3202, 3270), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(4860, 4926), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(8054, 8121), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(4930, 4995), match='You died in The Gnomish Mines on dungeon level 6 >
<re.Match object; span=(952, 1020), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(3205, 3273), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(6899, 6966), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(2974, 3041), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(3098, 3165), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(3842, 3909), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(5941, 6006), match='You die

<re.Match object; span=(4941, 5008), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(7276, 7343), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(5390, 5456), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(3918, 3983), match='You died in The Gnomish Mines on dungeon level 5 >
<re.Match object; span=(5920, 5986), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(4589, 4656), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(5310, 5376), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(950, 1017), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(1922, 1987), match='You died in The Gnomish Mines on dungeon level 6 >
<re.Match object; span=(6063, 6130), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(1092, 1162), match='You sta

<re.Match object; span=(4513, 4578), match='You died in The Gnomish Mines on dungeon level 3 >
<re.Match object; span=(3792, 3859), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(4429, 4495), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(6586, 6650), match='You died in The Gnomish Mines on dungeon level 4 >
<re.Match object; span=(4294, 4360), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(4995, 5061), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(1482, 1548), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(4386, 4454), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(6114, 6181), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(4233, 4304), match='You starved in The Dungeons of Doom on dungeon le>
<re.Match object; span=(6155, 6222), match='You di

...

<re.Match object; span=(6057, 6127), match='You starved in The Dungeons of Doom on dungeon le>
<re.Match object; span=(5355, 5422), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(2984, 3051), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(5137, 5204), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(6163, 6230), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(5066, 5133), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(2993, 3057), match='You died in The Gnomish Mines on dungeon level 3 >
<re.Match object; span=(950, 1017), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(5756, 5823), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(6386, 6453), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(7834, 7901), match='You die

<re.Match object; span=(949, 1016), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(6765, 6835), match='You starved in The Dungeons of Doom on dungeon le>
<re.Match object; span=(4447, 4512), match='You died in The Gnomish Mines on dungeon level 5 >
<re.Match object; span=(3940, 4007), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(960, 1027), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(5424, 5491), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(5382, 5447), match='You died in The Gnomish Mines on dungeon level 6 >
<re.Match object; span=(6957, 7024), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(4934, 5001), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(5548, 5615), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(3678, 3746), match='You died

<re.Match object; span=(6609, 6676), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(5653, 5720), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(5379, 5454), match='You were poisoned in The Dungeons of Doom on dung>
<re.Match object; span=(3682, 3749), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(7050, 7117), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(3178, 3245), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(3477, 3545), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(7527, 7594), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(4804, 4872), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(6417, 6484), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(4787, 4855), match='You di

<re.Match object; span=(6849, 6916), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(5218, 5284), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(4406, 4473), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(7665, 7732), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(4506, 4573), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(5046, 5113), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(6870, 6937), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(4087, 4153), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(955, 1022), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(5742, 5809), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(4730, 4795), match='You die

<re.Match object; span=(5178, 5244), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(3865, 3935), match='You starved in The Dungeons of Doom on dungeon le>
<re.Match object; span=(7262, 7332), match='You starved in The Dungeons of Doom on dungeon le>
<re.Match object; span=(3805, 3872), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(971, 1038), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(3160, 3227), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(4599, 4667), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(3816, 3883), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(6417, 6484), match='You died in The Dungeons of Doom on dungeon level>
<re.Match object; span=(4670, 4734), match='You died in The Gnomish Mines on dungeon level 3 >
<re.Match object; span=(3732, 3800), match='You die

In [22]:
with open(PATH_OUT, 'wb') as f:
    df = pd.DataFrame(rows)
    table = pa.Table.from_pandas(df)
    pq.write_table(table, f)

In [23]:
df = pq.read_table(PATH_OUT).to_pandas()
df

,path,frames,kills,role,race,gender,alignment,death_type,branch,depth,score,gold,steps,explevel,maxhp,killer
0,/Users/thayer/neurips-2021-nethack-starter-kit...,1629,96,Priest/Priestess,elf,male,chaotic,died,The Dungeons of Doom,2.0,1453.0,3.0,7468.0,6.0,48.0,a little dog
1,/Users/thayer/neurips-2021-nethack-starter-kit...,120,31,Priest/Priestess,human,female,neutral,died,The Dungeons of Doom,2.0,315.0,5.0,3094.0,3.0,24.0,a wand
2,/Users/thayer/neurips-2021-nethack-starter-kit...,119,32,Priest/Priestess,elf,male,chaotic,died,The Dungeons of Doom,3.0,349.0,10.0,2870.0,1.0,8.0,the wrath of Huhetotl
3,/Users/thayer/neurips-2021-nethack-starter-kit...,930,46,Tourist,human,male,neutral,died,The Dungeons of Doom,2.0,277.0,965.0,4245.0,3.0,35.0,a coyote
4,/Users/thayer/neurips-2021-nethack-starter-kit...,78,6,Healer,gnome,male,neutral,died,The Dungeons of Doom,1.0,11.0,1731.0,457.0,1.0,12.0,a bolt of cold
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3260,/Users/thayer/neurips-2021-nethack-starter-kit...,1971,126,Caveperson,human,female,lawful,starved,The Dungeons of Doom,3.0,1328.0,22.0,10037.0,5.0,58.0,None
3261,/Users/thayer/neurips-2021-nethack-starter-kit...,836,58,Archeologist,gnome,female,neutral,died,The Dungeons of Doom,4.0,599.0,58.0,4195.0,4.0,30.0,a giant bat
3262,/Users/thayer/neurips-2021-nethack-starter-kit...,1823,88,Archeologist,human,male,lawful,died,The Dungeons of Doom,3.0,689.0,23.0,6713.0,2.0,20.0,a couatl of Quetzalcoatl
3263,/Users/thayer/neurips-2021-nethack-starter-kit...,1036,32,Priest/Priestess,elf,female,chaotic,died,The Dungeons of Doom,1.0,102.0,11.0,2804.0,1.0,18.0,the wrath of Huhetotl


In [15]:
bad = df[(df['death_type'] != 'died') & (df['death_type'] != 'starved')]

In [16]:
bad

,path,frames,kills,role,race,gender,alignment,death_type,branch,depth,score,gold,steps,explevel,maxhp,killer
210,/Users/thayer/neurips-2021-nethack-starter-kit...,85,10,Caveperson,dwarf,female,lawful,poisoned,The Dungeons of Doom,1.0,43.0,7.0,744.0,1.0,18.0,None
231,/Users/thayer/neurips-2021-nethack-starter-kit...,1778,118,Samurai,human,male,lawful,poisoned,The Dungeons of Doom,9.0,4358.0,144.0,8398.0,7.0,70.0,None
549,/Users/thayer/neurips-2021-nethack-starter-kit...,1866,179,Barbarian,orc,male,chaotic,stone,The Gnomish Mines,8.0,6776.0,731.0,7875.0,8.0,91.0,None
678,/Users/thayer/neurips-2021-nethack-starter-kit...,3480,213,Samurai,human,male,lawful,poisoned,The Gnomish Mines,4.0,4986.0,53.0,16661.0,7.0,57.0,None
816,/Users/thayer/neurips-2021-nethack-starter-kit...,1160,150,Valkyrie,dwarf,female,lawful,poisoned,The Dungeons of Doom,4.0,1918.0,0.0,9844.0,6.0,61.0,None
842,/Users/thayer/neurips-2021-nethack-starter-kit...,947,103,Valkyrie,dwarf,female,lawful,poisoned,The Dungeons of Doom,4.0,2141.0,30.0,6756.0,6.0,72.0,None
849,/Users/thayer/neurips-2021-nethack-starter-kit...,1945,201,Priest/Priestess,human,female,lawful,poisoned,The Gnomish Mines,5.0,2563.0,25.0,12252.0,6.0,37.0,None
864,/Users/thayer/neurips-2021-nethack-starter-kit...,946,0,Barbarian,orc,female,chaotic,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None
918,/Users/thayer/neurips-2021-nethack-starter-kit...,119,56,Rogue,human,male,chaotic,poisoned,The Dungeons of Doom,1.0,126.0,11.0,2343.0,2.0,17.0,None
1120,/Users/thayer/neurips-2021-nethack-starter-kit...,963,26,Valkyrie,dwarf,female,lawful,poisoned,The Dungeons of Doom,1.0,71.0,7.0,2671.0,1.0,18.0,None


In [10]:
bad['path'].iloc[1]

'/Users/thayer/neurips-2021-nethack-starter-kit/nle_data/20210702-163442_at72fu1c/nle.82795.799.ttyrec.bz2'